In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, utils
from dataset import datasetClass
from pathlib import Path
from settingReader import settingReader


In [3]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)

resnext50_32x4d = models.resnext50_32x4d(pretrained=True)


# Do not forget to enable GPUs!

In [4]:
def use_gpu():
    torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available() 
                                                         else torch.FloatTensor)
use_gpu()

# You might want to use custom transforms

In [5]:
transform = transforms.Compose([transforms.ToTensor()])

# Download the dataset

In [6]:

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=False, transform=transform)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=False, transform=transform)
trainset = datasetClass(root='./data', train=True,download=True, transform=transform)

testset = datasetClass(root='./data', train=True,download=True, transform=transform)
print(trainset.targets)

settingInfo = settingReader()
# classess = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
classess = settingInfo.getItem("classNames")

hello
Files already downloaded and verified
hello
Files already downloaded and verified
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,

In [7]:
print(len(trainset))
print(len(testset))


1115
1115


In [8]:

restore = False #Whether to load the model from a file
lossArray = []
epochArray = []

def train(model, train_loader, optimizer, epoch):
    model.train()
    # problem in the following for loop
    for batch_idx, (data, target) in enumerate(train_loader):
        print(batch_idx)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {}, Loss: {:.3f}'.format(epoch, loss.item()))
    lossArray.append(loss.item())
    epochArray.append(epoch)


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, pred = output.max(1)
            test_loss += criterion(output, target).item()

            correct += pred.eq(target).sum().item()
            total += target.size(0)
    test_loss /= len(test_loader.dataset)

    print('Average loss: {:.3f}, Test Acc: {:.3f} ({}/{})'.format(test_loss, 100.*correct/total, correct, total))
    
    return test_loss


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(4,4), padding=1), #used to be 4 by 4
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(2048, 1024), #used to be 18432, 3072
            nn.ReLU(),
            nn.Linear(1024, 512), #used to be 3072, 1024
            nn.ReLU(),
            nn.Linear(512, 256),  #used to be 1024, 512
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256, 20),#usded to be 512,20
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(20, 5),#usded to be 512,20
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(5, 2)#usded to be 512,20

        )

    def forward(self, x):
        if torch.cuda.is_available():
           x = x.to(device="cuda")
        # conv layers
        x = self.conv_layer(x)
        # print("One")
        # print(x.size())
        x = x.view(x.size(0), -1)
        # print("Two")
        # print(x.size())


        # fc layer
        
        x = self.fc_layer(x)

        return x
batch_size = settingInfo.getItem("batchSize")
test_batch_size = 100
epochs = settingInfo.getItem("epochs")
lr = 1e-3
criterion = nn.CrossEntropyLoss()

#training
print(Path.cwd())
path = (Path.cwd() / "path.pth").resolve() #Does not work all the time. May need a direct file path. For mac, something like path = Path("/Users/userName/Desktop/ML/path.pth")
print(os.path.isfile(path))
model = CNN()
if os.path.isfile(path) and restore == True:
    model.load_state_dict(torch.load(path))
    model.eval()
optimizer = optim.Adam(model.parameters(), lr=lr)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=False)
losses = []
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    losses.append(test(model, val_loader))

torch.save(model.state_dict(), path)




mod_path = Path("/Volumes/Sans/ROI")

finalDict = dict()
finalDict['epochs'] = epochArray
finalDict['losses'] = lossArray

with open((mod_path / 'data.pickle').resolve(),'wb') as handle:
    pickle.dump(finalDict, handle, protocol = pickle.HIGHEST_PROTOCOL) 






/Users/games/Desktop/ML Github
True
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
Train Epoch: 1, Loss: 0.618
Average loss: 0.009, Test Acc: 15.067 (168/1115)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
3

NameError: name 'pickle' is not defined

In [11]:
import pickle
print(mod_path)
with open((mod_path / 'data.pickle').resolve(),'wb') as handle:
    pickle.dump(finalDict, handle, protocol = pickle.HIGHEST_PROTOCOL) 

/Volumes/Sans/ROI
